The purpose of the project is to learn the mapping from polar coordinates to a a discrete 10x10 grid of cells in the plane, using a neural network. 

The supervised dataset is given to you in the form of a generator (to be considered as a black box).

The model must achieve an accuracy of 95%, and it will be evaluated in a way **inversely proportional to the number of its parameters: the smaller, the better.**

**WARNING**: Any solution taking advantage of meta-knowledge about the generator will be automatically rejected.

In [62]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Reshape, concatenate
from tensorflow.keras.models import Model

Here is the generator. It returns triples of the form ((theta,rho),out) where (theta,rho) are the polar coordinates of a point in the first quadrant of the plane, and out is a 10x10 map with "1" in the cell corresponding to the point position, and "0" everywhere else.

By setting flat=True, the resulting map is flattened into a vector with a single dimension 100. You can use this variant, if you wish. 

In [63]:
def polar_generator(batchsize,grid=(10,10),noise=.002,flat=False):
  while True:
    x = np.random.rand(batchsize)
    y = np.random.rand(batchsize)
    out = np.zeros((batchsize,grid[0],grid[1]))
    xc = (x*grid[0]).astype(int)
    yc = (y*grid[1]).astype(int)
    for b in range(batchsize):
      out[b,xc[b],yc[b]] = 1
    #compute rho and theta and add some noise
    rho = np.sqrt(x**2+y**2) + np.random.normal(scale=noise)
    theta = np.arctan(y/np.maximum(x,.00001)) + np.random.normal(scale=noise)
    if flat:
      out = np.reshape(out,(batchsize,grid[0]*grid[1]))
    yield ((theta,rho),out)

Let's create an instance of the generator on a grid with dimension 3x4

In [64]:
g1,g2 = 3,4
gen = polar_generator(1,grid=(g1,g2),noise=0.0)

And now let's see a few samples.

In [65]:
(theta,rho),maps = next(gen)
for i,map in enumerate(maps):
  #let us compute the cartesian coordinates
  x = np.cos(theta[i])*rho[i]
  y = np.sin(theta[i])*rho[i]
  print("x coordinate (row): {}".format(int(x*g1)))
  print("y coordinate (col): {}".format(int(y*g2)))
  print("map:")
  print(np.reshape(map,(g1,g2)))

x coordinate (row): 1
y coordinate (col): 3
map:
[[0. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 0.]]


Exercise: add noise to the generator, and check the effect on the "ground truth".

# What to deliver

For the purposes of the project you are supposed to work with the **default 10x10 grid, and the default noise=.002**

The generator must be treatead as a black box, do not tweak it, and do not exploit its semantics that is supposed to be unknown. You are allowed to work with the "flat" modality, if you prefer so.

You need to:
1.   define an accuracy function (take inspiration from the code of the previous cell)
2.   define a neural network taking in input theta and rho, and returning out
3. measure the network's accuracy that must be above 95% (accuracy must be evaluated over at least 20000 samples)
4. tune the network trying to decrease as much as possible the numer of parameters, preserving an accuracy above 95%. Only your best network must be delivered.

You must deliver a SINGLE notebook working on colab, containing the code of the network, its summary, the training history, the code for the accurary metric and its evaluation on the network.

**N.B.** The accuracy must be above 95% but apart from that it does not influence the evaluation. You score will only depend on the number of parameters: the lower, the better.

#Good work!





In [98]:
from keras.models import Model
from keras.layers import Input, Concatenate, Dense, Activation, Lambda
import keras.backend as K

def create_model():
    # Define the input layers
    theta_input = Input(shape=(1,), name='theta_input')
    rho_input = Input(shape=(1,), name='rho_input')

    # Concatenate the inputs
    concatenated = Concatenate()([theta_input, rho_input])
    hidden=Dense(10,  activation="relu")(concatenated)
    hidden=Dense(10,  activation="relu")(hidden)

    # Define the output layer
    output = Dense(100, activation='softmax', name='output')(hidden)


    # Define the model
    model = Model(inputs=[theta_input, rho_input], outputs=output)
    
    return model

In [95]:
import tensorflow as tf

def custom_loss(y_true, y_pred):
    # Remove the extra dimension from the predicted output
    y_pred = tf.squeeze(y_pred, axis=1)

    # Compute the mean squared error loss
    loss = tf.reduce_mean(tf.square(y_true - y_pred))

    return loss

In [99]:
model=create_model()
# from tensorflow import keras
opt = keras.optimizers.Adam(learning_rate=0.05)
model.compile(optimizer=opt, loss="binary_crossentropy", metrics=["accuracy"])
model.summary()

ValueError: ignored

In [80]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Define the EarlyStopping callback
early_stop = EarlyStopping(monitor='loss', patience=5, verbose=1, mode='auto')

# Define the ReduceLROnPlateau callback
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.2, patience=3, verbose=1, mode='auto', min_lr=0.0001)

In [91]:
# Define the generator instance
train_gen = polar_generator(batchsize=32, flat=True)

# Train the model using the generator
history=model.fit_generator(train_gen, steps_per_epoch=2000, epochs=100, callbacks=[early_stop, reduce_lr])

Epoch 1/100


<ipython-input-91-abfc9341abd8>:5: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history=model.fit_generator(train_gen, steps_per_epoch=2000, epochs=100, callbacks=[early_stop, reduce_lr])


ValueError: ignored

In [ ]:
gen=polar_generator(batchsize=1)
dat=next(gen)
m=model.predict(dat[0])
print(m)
print(dat[1])

In [ ]:
import matplotlib.pyplot as plt

# Plot the training loss
plt.plot(history.history['loss'])
plt.title('Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()

# Plot the training accuracy
plt.plot(history.history['accuracy'])
plt.title('Training Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.show()